In [1]:
from ultralytics import YOLO
model = YOLO("../data/runs/classify/train/weights/best.pt")

In [7]:
model.val(data="../data/img/techniques", project="../data/runs/classify")

Ultralytics 8.3.78 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA TITAN RTX, 24576MiB)
train: /mnt/c/Projects/climbing-technique-detector/data/img/techniques/train... found 558 images in 4 classes ✅ 
val: /mnt/c/Projects/climbing-technique-detector/data/img/techniques/val... found 128 images in 4 classes ✅ 
test: /mnt/c/Projects/climbing-technique-detector/data/img/techniques/test... found 116 images in 4 classes ✅ 


val: Scanning /mnt/c/Projects/climbing-technique-detector/data/img/techniques/val... 128 images, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.65it/s]


                   all       0.68          1
Speed: 0.9ms preprocess, 5.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to ../data/runs/classify/val


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f96b22b08c0>
curves: []
curves_results: []
fitness: 0.83984375
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.6796875, 'metrics/accuracy_top5': 1.0, 'fitness': 0.83984375}
save_dir: PosixPath('../data/runs/classify/val')
speed: {'preprocess': 0.928396773588247, 'inference': 5.234049851537748, 'loss': 0.03798950785949273, 'postprocess': 0.0038443439507318544}
task: 'classify'
top1: 0.6796875
top5: 1.0